In [ ]:
from google.colab import drive

In [ ]:
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
from keras.layers import concatenate
from keras.layers import GlobalAveragePooling2D
from keras.utils import plot_model
from keras.layers import Input
from keras.models import Sequential
from keras.utils import to_categorical


In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import scipy.misc
from skimage import io
import pandas as pd
import os
from scipy.ndimage import gaussian_filter

In [ ]:
def square_padding (image):
  original_height, original_width = image.shape[0], image.shape[1]

  # Calculate the amount of padding needed to make the image a square
  pad_height = max(0, original_width - original_height)
  pad_width = max(0, original_height - original_width)

  # Pad the image with 0s to make it a square
  padded_image = np.pad(image, ((0, pad_height), (0, pad_width), (0, 0)), 'constant')
  return padded_image

In [ ]:
def find_meanAndVaraince(arr):
  tensor = tf.convert_to_tensor(arr)

# Compute the mean and variance of the Tensor
  meanVal = tf.math.reduce_mean(tensor)
  varianceVal = tf.math.reduce_variance(tensor)
  return [meanVal,varianceVal]

In [ ]:
def training_image_reader(csv_file, image_dir, trainingCond,output_shape = (224,224)):
    data = pd.read_csv(csv_file)
    mrnNumbers = data.iloc[:,0]
    eyePosition =  data.iloc[:,1]
    gestational_Age = data.iloc[:,3]
    weights = data.iloc[:,4]
    zoneValue = data.iloc[:,6]
    print('zoneValue')
    print(zoneValue)
    print('end')
    stageValue = data.iloc[:,7]
    PlusValue = data.iloc[:,8]
    
    num_of_mrns = len(mrnNumbers)
    
    FAImages = []
    fundusImages = []
    age = []
    weight = []

    zone = []
    stage= []
    plus= []
    maxm =0;
    for idx in range(0,num_of_mrns):
       print('idx=',idx)
       pathFA = str(mrnNumbers[idx])+'/'+'FALeft'
       pathFundus = str(mrnNumbers[idx])+'/'+'fundusLeft'
       if eyePosition[idx]=='R':
        pathFA = str(mrnNumbers[idx])+'/'+'FARight'
        pathFundus = str(mrnNumbers[idx])+'/'+'fundusRight'
       pathFA = os.path.join(image_dir,pathFA)
       pathFundus = os.path.join(image_dir,pathFundus)
       #print(pathFA)
       #print(pathFundus)
       for images1 in os.listdir(pathFA):
          #if maxm == 10:
           #break
          if (images1.endswith(".jpg")):
            for images2 in os.listdir(pathFundus):
              #if maxm == 10:
               # break
              if (images2.endswith(".jpg")):
                #print(images1)
                #print(images2)
                pathImages1 = os.path.join(pathFA+'/'+images1)
                pathImages2 = os.path.join(pathFundus+'/'+images2)
                #print(pathImages1)
                #print(pathImages2)
                faImage = cv2.imread(pathImages1)
                fundusImage = cv2.imread(pathImages2)
                #print(faImage)
                #print(fundusImage)
                faImage1 = square_padding(faImage)
                faImage1 = cv2.resize(faImage, output_shape)
               
                fundusImage1 = square_padding(fundusImage)
                fundusImage1 = cv2.resize(fundusImage1, output_shape)
                fundusImages.append(fundusImage1)
                FAImages.append(fundusImage1)
                age.append(gestational_Age[idx])
                weight.append(weights[idx])
                zone.append(zoneValue[idx])
                stage.append(stageValue[idx])
                plus.append(PlusValue[idx])  
                maxm = maxm+1              
                                            
    FAImages = np.array(FAImages)
    FAImages =FAImages/255
    fundusImages = np.array(fundusImages)
    fundusImages = fundusImages/255
    age = np.array(age)
    weight = np.array(weight)
    zone = np.array(zone)
    
    print('zoneee')
    print(zone)
    print('endl')
    stage = np.array(stage)
    
    #if trainingCond == 'training':
    zone = to_categorical(zone, num_classes=4)
    stage = to_categorical(stage, num_classes=5)
    print('zoneee')
    print(zone)
    print('endl')
    
    plus = np.array(plus)
    print(stage.shape)
    dataset = {'FAImages': FAImages, 'fundusImages': fundusImages, 'age':age, 'weight': weight, 'zone':zone, 'stage':stage, 'plus':plus}  
    print(len(plus))
    return dataset

In [ ]:
dataloader = training_image_reader('/content/drive/MyDrive/Data/Train/ROPTrainingData.csv', '/content/drive/MyDrive/Data/Train/Images','training', output_shape = (224,224))


In [ ]:
import json

In [ ]:
import pickle

In [ ]:
data = pickle.dumps(dataloader)

# Write the byte stream to a file
with open('dataset.pkl', 'wb') as f:
  f.write(data)

In [ ]:
one_weight=0.5
zero_weight=0.5
from keras import backend as ke

In [ ]:
def weighted_binary_crossentropy(y_true, y_pred):
  b_ce = ke.binary_crossentropy(y_true, y_pred)

        # weighted calc
  weight_vector = y_true * one_weight + (1 - y_true) * zero_weight
  weighted_b_ce = weight_vector * b_ce
  return ke.mean(weighted_b_ce)

In [ ]:
FAImages = dataloader['FAImages']
fundusImages = dataloader['fundusImages']
age = dataloader['age']

meanAge = 25.584545454545456
varianceAge = 1.850775580110642
age = (age - meanAge) / varianceAge
weight = dataloader['weight']
meanWeight = 0.6219090909090909
varianceWeight = 0.12644111424651727
weight = (weight-meanWeight)/varianceWeight
zone = dataloader['zone']
stage = dataloader['stage']
plus = dataloader['plus']
medical_data = np.stack((age,weight), axis = 1)

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
# first input model
model1 = Sequential()
model1.add(Dense(2, input_dim=2, activation="relu"))
#model1.add(Normalization(mean=0.5, variance=1.0))
model1.add(Dense(4, activation="relu"))

#Second input model
visible2 = Input(shape=(224,224,3))
conv_base2 = VGG16(include_top=False,weights='imagenet', input_shape=IMAGE_SIZE + [3])
for layer in conv_base2.layers:
    layer._name = layer._name + str('_D')
conv_base2.trainable = False ## Not trainable weights
layerFirst = GlobalAveragePooling2D()(conv_base2.output)
flat2 = Flatten(name="flatten1")(layerFirst)
flat2 = Dense(256, activation = "relu")(flat2)
flat2 = Dense(64, activation = "relu")(flat2)
flat2 = Dense(16, activation = "relu")(flat2)

# third input model

conv_base3 = VGG16(include_top=False,weights='imagenet', input_shape=IMAGE_SIZE + [3])
conv_base3._name="NAME"
conv_base3.trainable = False ## Not trainable weights
layerSecond = GlobalAveragePooling2D()(conv_base3.output)
flat3 = Flatten(name="flatten2")(layerSecond )
flat3 = Dense(256, activation = "relu")(flat3)
flat3 = Dense(64, activation = "relu")(flat3)
flat3 = Dense(16, activation = "relu")(flat3)

mergeImages = concatenate([flat2,flat3],name="Images_Combine")
flatImages = Dense(64, activation = "relu")(mergeImages)
flatImages = Dense(16, activation = "relu")(flatImages)

# merge input models
merges = concatenate([model1.output,flatImages],name="concatenated_layer")
#flatImage = Dense(16, activation = "relu")(mergeImage)


# interpretation model
top_model = Dense(64, activation = "relu")(merges)
top_model = Dropout(rate=0.3)(top_model)
dense = Dense(16, activation = "relu")(top_model)

zone_pred = Dense(4, activation = 'softmax', name='zone_Prediction')(dense)
plus_pred = Dense(1, activation = 'sigmoid', name='Plus_Prediction')(dense)
stage_pred = Dense(5, activation = 'softmax', name='stage_Prediction')(dense)

model = Model(inputs=[model1.input, conv_base2.input, conv_base3.input], outputs=[stage_pred,zone_pred,plus_pred])
# summarize layers
print(model.summary())

# plot model
plot_model(model)



In [ ]:
def weighted_bincrossentropy(true, pred, weight_zero = 1, weight_one = 1):
    """
    Calculates weighted binary cross entropy. The weights are fixed.
        
    This can be useful for unbalanced catagories.
    
    Adjust the weights here depending on what is required.
    
    For example if there are 10x as many positive classes as negative classes,
        if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives 
        will be penalize 10 times as much as false negatives.
    """
    true = tf.cast(true, dtype=tf.float32)
    # calculate the binary cross entropy
    bin_crossentropy = ke.binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weighted_bin_crossentropy = weights * bin_crossentropy 

    return ke.mean(weighted_bin_crossentropy)

In [ ]:
model.compile(optimizer='adam', 
              loss={
                  'zone_Prediction': 'categorical_crossentropy', 
                  'stage_Prediction': 'categorical_crossentropy', 
                  'Plus_Prediction': weighted_bincrossentropy},
              metrics={
                  'zone_Prediction': 'accuracy', 
                  'stage_Prediction': 'accuracy',
                  'Plus_Prediction': 'accuracy'})

In [ ]:
print(stage.dtype)
print(zone.dtype)
plus.astype(int)
print(plus.dtype)

float32
float32
int64


In [ ]:
loss1=[]
loss2=[]
loss3=[]
accuracy1=[]
accuracy2=[]
accuracy3=[]
loss=[]
from google.colab.patches import cv2_imshow

In [ ]:
lr = 0.0001
num_epochs = 50
batch_size = 10
optimizer_affine = tf.keras.optimizers.Adam(learning_rate=lr)  

In [ ]:

num_epochs =10
batch_size =10
for epoch in range(1,num_epochs+1):
  shape1 = FAImages.shape
  print(shape1)
  num_of_images = shape1[0]
  print('num_of_images',num_of_images)
  num_of_batches = int( num_of_images/batch_size)
  print('num of Batches=', num_of_batches)
  for idx in range(0,num_of_batches):
    batch_idx = np.random.randint(num_of_images, size=batch_size)
    FAImagesBatch = FAImages[batch_idx, :]
    fundusImagesBatch = fundusImages[batch_idx,:]
    ageBatch = age[batch_idx]
    zoneBatch = zone[batch_idx]
    stageBatch = stage[batch_idx]
    plusBatch = plus[batch_idx]
    medical_dataBatch = medical_data[batch_idx]
    loss= model.train_on_batch([medical_dataBatch,FAImagesBatch,fundusImagesBatch],[stageBatch,zoneBatch,plusBatch])
    loss1.append(loss[1])
    loss2.append(loss[2])
    loss3.append(loss[3])
    accuracy1.append(loss[4])
    accuracy2.append(loss[5])
    accuracy3.append(loss[6])
    if idx%10==0:
      print('Epoch=',epoch,' Batch=',idx)
  print('Epoch=',epoch)

In [ ]:
model.save('model.h5')

In [ ]:
model = tf.keras.models.load_model('model.h5')

In [ ]:
dataloaderTest = training_image_reader('/content/drive/MyDrive/Data/Test/ROPtest3.csv', '/content/drive/MyDrive/Data/Test/Images','testing', output_shape = (224,224))


In [ ]:
FAImages = dataloaderTest['FAImages']
fundusImages = dataloaderTest['fundusImages']
age = dataloaderTest['age']
age = (age - meanAge) / varianceAge
#print(age)
weight = dataloaderTest['weight']
#print(weight)
weight = (weight-meanWeight)/(varianceWeight+1e-8)
#print('weight')
print(weight)
zone = dataloaderTest['zone']
stage = dataloaderTest['stage']
plus = dataloaderTest['plus']

medical_data = np.stack((age,weight), axis = 1)
shape1 = FAImages.shape
num_of_images = shape1[0]

In [ ]:
print(num_of_images)

16


In [ ]:
stagePreds=[]
zonePreds=[]
plusPreds=[]

In [ ]:

[stagePreds,zonePreds,plusPreds] = model([medical_data,FAImages,fundusImages])


In [ ]:
def findMaxCountVal(arr):
  counts = np.bincount(arr)
  max_count = counts.max()
  max_value = np.argmax(counts)
  return [max_value,max_count]

In [ ]:
zonePreds = np.argmax(zonePreds, axis=1)
stagePreds = np.argmax(stagePreds, axis=1)
print(zonePreds)
print(stagePreds)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [ ]:
stagePreds=np.array(stagePreds)
zonePreds = np.array(zonePreds)
plusPreds = np.array(plusPreds)
arr_size = len(plusPreds)
for x in range(0,arr_size):
  if plusPreds[x]<=0.5:
    plusPreds[x]=0
  else:
    plusPreds[x]=1
plusPreds = plusPreds.astype(np.int64)
plusPreds = plusPreds.flatten()
print(stagePreds)
print(plusPreds)
print(zonePreds)


In [ ]:


stageVal, stageMaxCount = findMaxCountVal(stagePreds)
zoneVal, zoneMaxCount = findMaxCountVal(zonePreds)
plusVal, plusMaxCount = findMaxCountVal(plusPreds)
stageValPer = (stageMaxCount/num_of_images)*100
zoneValPer = (zoneMaxCount/num_of_images)*100
plusValPer = (plusMaxCount/num_of_images)*100

In [ ]:
print('stage=',stage[0],' stagePred=',stageVal,' stageMaxCount=',stageMaxCount,' stageValPer=', stageValPer)
print('zone=',zone[0],' zonePred=',zoneVal,' zoneMaxCount=',zoneMaxCount,' zoneValPer=', zoneValPer)
print('plus=',plus[0],' plusPred=',plusVal,' plusMaxCount=',plusMaxCount,' plusValPer=', plusValPer)


stage= [0. 0. 1. 0. 0.]  stagePred= 2  stageMaxCount= 16  stageValPer= 100.0
zone= [0. 0. 0. 1.]  zonePred= 2  zoneMaxCount= 16  zoneValPer= 100.0
plus= 1  plusPred= 0  plusMaxCount= 12  plusValPer= 75.0


In [ ]:
import pickle

# write list to binary file
def write_list(a_list):
    # store list in binary file so 'wb' mode
    with open('listfile', 'wb') as fp:
        pickle.dump(names, fp)
        print('Done writing list into a binary file')

# Read list to memory
def read_list():
    # for reading also binary mode is important
    with open('sampleList', 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [ ]:
with open('loss1.pkl', 'wb') as f:
  # Pickle the list and write it to the file
  pickle.dump(loss1, f)

In [ ]:
with open('loss2.pkl', 'wb') as f:
  # Pickle the list and write it to the file
  pickle.dump(loss2, f)

In [ ]:
with open('loss3.pkl', 'wb') as f:
  # Pickle the list and write it to the file
  pickle.dump(loss3, f)

In [ ]:
with open('accuracy1.pkl', 'wb') as f:
  # Pickle the list and write it to the file
  pickle.dump(accuracy1, f)

In [ ]:
with open('accuracy2.pkl', 'wb') as f:
  # Pickle the list and write it to the file
  pickle.dump(accuracy2, f)

In [ ]:
with open('accuracy3.pkl', 'wb') as f:
  # Pickle the list and write it to the file
  pickle.dump(accuracy3, f)

In [ ]:
with open('loss1.pkl', 'rb') as f:
  # Unpickle the data and store it in a variable
  newloss1 = pickle.load(f)
print(newloss1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def new_array(array1) :
  n =len(array1)
  idx=0
  newArray=[]
  while(idx<n):
    if(idx%20==0):
     newArray.append(array1[idx])
    idx =idx+1
  return newArray

In [ ]:
accuracy11 = new_array(np.array(accuracy1))
accuracy22 = new_array(np.array(accuracy2))
accuracy33 = new_array(np.array(accuracy3))

In [ ]:
fig, ax = plt.subplots()

# Plot each list as a line
ax.plot(range(len(accuracy11)), accuracy11, label='Stage accuracy')
ax.plot(range(len(accuracy22)), accuracy22, label='Zone accuracy')
ax.plot(range(len(accuracy33)), accuracy33, label='Plus accuracy')

# Add a legend and show the plot
ax.legend()
plt.show()

In [ ]:
dataTest = pd.read_csv('/content/drive/MyDrive/Data/Data/ROPTrainingData.csv')
mrnNumbers = dataTest.iloc[:,0]
num_of_mrns = len(mrnNumbers)
eyePosition =  dataTest.iloc[:,1]
for idx in range(0,num_of_mrns):
  mrnNumber = mrnNumbers[idx]
  dataloader = testing_image_reader(mrnNumber,'/content/drive/MyDrive/Data/Data/ROPTrainingData.csv', '/content/drive/MyDrive/Data/Data/Images','training', output_shape = (224,224))
  FAImages = dataloaderTest['FAImages']
  fundusImages = dataloaderTest['fundusImages']
  age = dataloaderTest['age']
  age = (age - meanAge) / varianceAge
  #print(age)
  weight = dataloaderTest['weight']
  #print(weight)
  weight = (weight-meanWeight)/(varianceWeight+1e-8)
  #print('weight')
  print(weight)
  zone = dataloaderTest['zone']
  stage = dataloaderTest['stage']
  plus = dataloaderTest['plus']

  medical_data = np.stack((age,weight), axis = 1)
  shape1 = FAImages.shape
  num_of_images = shape1[0]
  print(num_of_images)
  stagePreds=[]
  zonePreds=[]
  plusPreds=[]
  [stagePreds,zonePreds,plusPreds] = model([medical_data,FAImages,fundusImages])
  zonePreds = np.argmax(zonePreds, axis=1)
  stagePreds = np.argmax(stagePreds, axis=1)
  stagePreds=np.array(stagePreds)
  zonePreds = np.array(zonePreds)
  plusPreds = np.array(plusPreds)
  arr_size = len(plusPreds)
  for x in range(0,arr_size):
    if plusPreds[x]<=0.5:
      plusPreds[x]=0
    else:
      plusPreds[x]=1
  plusPreds = plusPreds.astype(np.int64)
  plusPreds = plusPreds.flatten()
  print('mrnNumber=',mrnNumber)
  print('eyePosition=',eyePosition[idx])
  print(stagePreds)
  print(plusPreds)
  print(zonePreds)
  stageVal, stageMaxCount = findMaxCountVal(stagePreds)
  zoneVal, zoneMaxCount = findMaxCountVal(zonePreds)
  plusVal, plusMaxCount = findMaxCountVal(plusPreds)
  stageValPer = (stageMaxCount/num_of_images)*100
  zoneValPer = (zoneMaxCount/num_of_images)*100
  plusValPer = (plusMaxCount/num_of_images)*100
  print('stage=',stage[0],' stagePred=',stageVal,' stageMaxCount=',stageMaxCount,' stageValPer=', stageValPer)
  print('zone=',zone[0],' zonePred=',zoneVal,' zoneMaxCount=',zoneMaxCount,' zoneValPer=', zoneValPer)
  print('plus=',plus[0],' plusPred=',plusVal,' plusMaxCount=',plusMaxCount,' plusValPer=', plusValPer)


In [ ]:
def testing_image_reader(mrnNum,csv_file, image_dir, trainingCond,output_shape = (224,224)):
    data = pd.read_csv(csv_file)
    mrnNumbers = data.iloc[:,0]
    eyePosition =  data.iloc[:,1]
    gestational_Age = data.iloc[:,3]
    weights = data.iloc[:,4]
    zoneValue = data.iloc[:,6]
    #print('zoneValue')
    #print(zoneValue)
   # print('end')
    stageValue = data.iloc[:,7]
    PlusValue = data.iloc[:,8]
    
    num_of_mrns = len(mrnNumbers)
    
    FAImages = []
    fundusImages = []
    age = []
    weight = []

    zone = []
    stage= []
    plus= []
    maxm =0;
    for idx in range(0,num_of_mrns):
      if(mrnNumbers[idx] == mrnNum):

        pathFA = str(mrnNumbers[idx])+'/'+'FALeft'
        pathFundus = str(mrnNumbers[idx])+'/'+'fundusLeft'
        if eyePosition[idx]=='R':
         pathFA = str(mrnNumbers[idx])+'/'+'FARight'
         pathFundus = str(mrnNumbers[idx])+'/'+'fundusRight'
        pathFA = os.path.join(image_dir,pathFA)
        pathFundus = os.path.join(image_dir,pathFundus)
       #print(pathFA)
       #print(pathFundus)
        for images1 in os.listdir(pathFA):
           #if maxm == 10:
            #break
           if (images1.endswith(".jpg")):
             for images2 in os.listdir(pathFundus):
               #if maxm == 10:
               # break
               if (images2.endswith(".jpg")):
                #print(images1)
                #print(images2)
                 pathImages1 = os.path.join(pathFA+'/'+images1)
                 pathImages2 = os.path.join(pathFundus+'/'+images2)
                #print(pathImages1)
                #print(pathImages2)
                 faImage = cv2.imread(pathImages1)
                 fundusImage = cv2.imread(pathImages2)
                #print(faImage)
                #print(fundusImage)
                 faImage1 = square_padding(faImage)
                 faImage1 = cv2.resize(faImage, output_shape)
                 FAImages.append(faImage1)
                 fundusImage1 = square_padding(fundusImage)
                 fundusImage1 = cv2.resize(fundusImage1, output_shape)
                 fundusImages.append(fundusImage1)
                 age.append(gestational_Age[idx])
                 weight.append(weights[idx])
                 zone.append(zoneValue[idx])
                 stage.append(stageValue[idx])
                 plus.append(PlusValue[idx])  
                 maxm = maxm+1              
                                            
    FAImages = np.array(FAImages)
    FAImages =FAImages/255
    fundusImages = np.array(fundusImages)
    fundusImages = fundusImages/255
    age = np.array(age)
    weight = np.array(weight)
    zone = np.array(zone)
    
    #print('zoneee')
    #print(zone)
    #print('endl')
    stage = np.array(stage)
    
    #if trainingCond == 'training':
    zone = to_categorical(zone, num_classes=4)
    stage = to_categorical(stage, num_classes=5)
   # print('zoneee')
   # print(zone)
   # print('endl')
    
    plus = np.array(plus)
   # print(stage.shape)
    dataset = {'FAImages': FAImages, 'fundusImages': fundusImages, 'age':age, 'weight': weight, 'zone':zone, 'stage':stage, 'plus':plus}  
    print(len(plus))
    return dataset